In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),  # Mean for each channel
                         (0.5, 0.5, 0.5))  # Std for each channel
])

# Load CIFAR-10 training and test datasets
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Define DataLoaders
batch_size = 128
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [ ]:
# data_iter = iter(train_loader)
# images, labels = next(data_iter)
# print("Batch shape:", images.shape)  # Shape will be [batch_size, 3, 32, 32]
# print("Batch labels:", labels.shape)

Batch shape: torch.Size([128, 3, 32, 32])
Batch labels: torch.Size([128])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define Encoder
encoder = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  # Output: (64, 32, 32)
    nn.BatchNorm2d(64),  #here we are using the (64) coming from teh output of above layer
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
)

decoder = nn.Sequential(
    nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.Tanh()  # output normalised between -1 to 1

)

# tried increaseing the layer but not getting good accuracy
encoder = encoder.to(device)
decoder = decoder.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.001)




In [ ]:
# Start of the training loop
num_epochs = 15
for epoch_idx in range(num_epochs):
    encoder.train()
    decoder.train()
    cumulative_loss = 0.0

    # Loop through each batch in the training set
    for batch_images, _ in train_loader:
        batch_images = batch_images.to(device)

        # Forward pass through the encoder and decoder
        encoded_images = encoder(batch_images)
        reconstructed_images = decoder(encoded_images)
        reconstruction_loss = criterion(reconstructed_images, batch_images)
        optimizer.zero_grad()
        reconstruction_loss.backward()
        optimizer.step()

        cumulative_loss += reconstruction_loss.item() #to accumuate the loss

    average_loss = cumulative_loss / len(train_loader)
    print(f"Epoch {epoch_idx + 1}/{num_epochs}, Average Loss: {average_loss:.4f}")

Epoch 1/15, Average Loss: 0.0427
Epoch 2/15, Average Loss: 0.0211
Epoch 3/15, Average Loss: 0.0180
Epoch 4/15, Average Loss: 0.0160
Epoch 5/15, Average Loss: 0.0151
Epoch 6/15, Average Loss: 0.0140
Epoch 7/15, Average Loss: 0.0135
Epoch 8/15, Average Loss: 0.0129
Epoch 9/15, Average Loss: 0.0124
Epoch 10/15, Average Loss: 0.0122
Epoch 11/15, Average Loss: 0.0117
Epoch 12/15, Average Loss: 0.0117
Epoch 13/15, Average Loss: 0.0111
Epoch 14/15, Average Loss: 0.0111
Epoch 15/15, Average Loss: 0.0107


In [ ]:
# Feature Extraction
encoder.eval()
feat, lbls = [], []
with torch.no_grad():
    for imgs, lbl in train_loader:
        imgs = imgs.to(device)
        encoded_imgs = encoder(imgs)
        feat.append(encoded_imgs.view(encoded_imgs.size(0), -1).cpu())
        lbls.append(lbl)

feat = torch.cat(feat)
lbls = torch.cat(lbls)

# Defining Classifier
input_size = feat.size(1)
model = nn.Sequential(
    nn.Linear(input_size, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.1),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.1),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Dropout(0.1),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Dropout(0.1),
    nn.Linear(64, 10)  # Output layer for CIFAR-10 has 10 neurons as to classify the 10 classes
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training Function
def train(model, criterion, optimizer, loader, epoch_num):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for features, labels in loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch_num + 1}, Loss: {total_loss / len(loader)}")


In [ ]:
# Test Function
def test(model, criterion, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

In [ ]:
dataset = torch.utils.data.TensorDataset(feat, lbls)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training the model
for epoch in range(15):
    train(model, criterion, optimizer, loader, epoch)


Epoch 1, Loss: 0.5632164389885905
Epoch 2, Loss: 0.5382214123788087
Epoch 3, Loss: 0.5004636030977644
Epoch 4, Loss: 0.48134150133108544
Epoch 5, Loss: 0.4584439420486655
Epoch 6, Loss: 0.43881675863967223
Epoch 7, Loss: 0.4125533736956394
Epoch 8, Loss: 0.3967189257940673
Epoch 9, Loss: 0.38395664743755176
Epoch 10, Loss: 0.3649588644199664
Epoch 11, Loss: 0.35212505923207765
Epoch 12, Loss: 0.3405344793406289
Epoch 13, Loss: 0.32403787814290325
Epoch 14, Loss: 0.3132938877548403
Epoch 15, Loss: 0.29897187997008223
Test Accuracy: 95.32%


95.32

In [ ]:
# Testing the model
test(model, criterion, loader)

Test Accuracy: 95.32%


95.32